## Project Roots Market Survey (Engineering)

The aim of this code is to engineer and cleanse the **Project Roots Market Survey**, transforming a wide data set into a tall data set enriched with useful meta data and primed for data analysis.

### Import packages

In [1]:
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

import os
import pickle
from datetime import date, datetime

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import numpy as np

### Note: 
If using a new environment, pip install the following from terminal:\
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request

### Create function to connect to Google Sheets API
Used walkthrough to connect to and read from Google Sheet\
https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c

In [3]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']  

def connectGSheet(gsheet_id, gsheet_range):
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Specify json file generated using this tutorial:
            # https://developers.google.com/sheets/api/quickstart/python
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES) 
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Google Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=gsheet_id,
                                range=gsheet_range).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

**Read sheet:** form_response

In [4]:
# Define google sheet id and range
gsheet_id = '1YDodYS7JAV7R0icO0s28i9UGKHO2LbOcbfCYjmhvSmI'
gsheet_range = 'form_response!A1:X201'      
    
# Function call   
connectGSheet(gsheet_id, gsheet_range)

# Generate data frame of raw survey response data
df_rd = pd.DataFrame(values_input[1:], columns=values_input[0])

**Read sheet:** question_meta_data

In [5]:
# Define google sheet range
gsheet_range = 'question_meta_data!A1:F23'      
    
# Function call   
connectGSheet(gsheet_id, gsheet_range)

# Generate data frame of meta data pertaining to survey questions
df_md = pd.DataFrame(values_input[1:], columns=values_input[0])

**Read sheet:** Q18

In [6]:
# Define google sheet id and range
gsheet_range = 'Q18!A1:B14'      
    
# Function call   
connectGSheet(gsheet_id, gsheet_range)

# Generate data frame of raw survey response data
df_q18 = pd.DataFrame(values_input[1:], columns=values_input[0])

**Read sheet:** Q20

In [7]:
# Define google sheet id and range
gsheet_range = 'Q20!A1:B46'      
    
# Function call   
connectGSheet(gsheet_id, gsheet_range)

# Generate data frame of raw survey response data
df_q20 = pd.DataFrame(values_input[1:], columns=values_input[0])

### Merge survey  data with updated Q18 & Q20 answers
Replace free text responses with manually updated answers that conform to the naming convention outlined in earlier questions (e.g. UK -> United Kingdom, Holland -> Netherlands etc.)

In [8]:
# Left join between raw dataset and the Q18 manual categorisation table
df_rd = pd.merge(df_rd, df_q18, how='left', left_on='Please list the rest of your ethnicities here if you have more than 3', right_on='Please list the rest of your ethnicities here if you have more than 3')

In [9]:
# Left join between raw dataset and the Q20 manual categorisation table
df_rd = pd.merge(df_rd, df_q20, how='left', left_on='Please add any other countries you grew up in', right_on='Please add any other countries you grew up in')

In [10]:
# SENSE CHECK: Total rows should be 200
df_rd.shape[0]

200

In [11]:
# Re-order new columns 'Cleansed (Q18)' and '(Cleansed (Q20)')
col = df_rd.pop('Cleansed (Q18)')
df_rd.insert(6, col.name, col)

col = df_rd.pop('Cleansed (Q20)')
df_rd.insert(9, col.name, col)

In [12]:
# Rename cleansed columns
df_rd = df_rd.rename(columns={'Cleansed (Q18)':'Please list the rest of your ethnicities here if you have more than 3 (Cleansed)','Cleansed (Q20)':'Please add any other countries you grew up in (Cleansed)'})

In [13]:
# Fill NaN values
df_rd = df_rd.fillna('')

In [14]:
# Concatenate multiple columns to get a single list of all countries
df_rd["CF Total ethnicities (List)"] = df_rd[['What country is your first ethnic background? ', 'What country is your second ethnic background? (If you have multiple)','What country is your third ethnic background? (If you have multiple)','Please list the rest of your ethnicities here if you have more than 3 (Cleansed)']].agg(', '.join, axis=1)

# Remove additional ' ,' generated in previous step
df_rd["CF Total ethnicities (List)"] = df_rd["CF Total ethnicities (List)"].str.replace(r'(,\s*)+', ', ', regex=True).str.strip(', ')

# Calculate number of countries supplied for ethnic background
df_rd["CF Total ethnicities (Number)"] = df_rd["CF Total ethnicities (List)"].str.split(',').apply(lambda x: len([item for item in x if item.strip() != '']))

# Concatenate multiple columns to get a single list of all countries
df_rd["CF Total countries grown up in (List)"] = df_rd[['What country did you grow up in primarily? ','Please add any other countries you grew up in (Cleansed)']].agg(', '.join, axis=1)

# Remove additional ' ,' generated in previous step
df_rd["CF Total countries grown up in (List)"] = df_rd["CF Total countries grown up in (List)"].str.replace(r'(,\s*)+', ', ', regex=True).str.strip(', ')

# Calculate number of countries supplied for ethnic background
df_rd["CF Total countries grown up in (Number)"] = df_rd["CF Total countries grown up in (List)"].str.split(',').apply(lambda x: len([item for item in x if item.strip() != '']))

### Create tall data set with new naming convention
Certain data analysis is prefereable with a tall data set, therefore we will add a 'tall' data set sheet to our final GSheet.

In [15]:
# Create unique identifier and pad with leading zeroes
df_rd['recipient_id'] = (df_rd.index + 1).astype(str).str.zfill(3)

In [16]:
# Re-order spreadsheet and bring id column to front
col = df_rd.pop('recipient_id')
df_rd.insert(0, col.name, col)

In [17]:
# Rename 'Timestamp' to 'timestamp' to uphold naming convention
df_rd = df_rd.rename(columns={'Timestamp':'timestamp', 'Email? (join our future mailing list when we launch! 😁)':'email'})

In [18]:
# Use 'melt' syntax to unpivot data and transform wide data set to a tall data set
df_piv = pd.melt(df_rd, id_vars=
                 ['recipient_id','timestamp','email'],
                 value_vars=['Age?',
                             'What country is your first ethnic background? ',
                             'What country is your second ethnic background? (If you have multiple)',
                             'What country is your third ethnic background? (If you have multiple)',
                             'Please list the rest of your ethnicities here if you have more than 3',
                             'Please list the rest of your ethnicities here if you have more than 3 (Cleansed)',                                                                         
                             'What country did you grow up in primarily? ',
                             'Please add any other countries you grew up in',
                             'Please add any other countries you grew up in (Cleansed)',                                                                          
                             '1. Did you experience uncertainty or conflict between your family culture and the culture of the country you grew up in?',
                             '2.  What kind of positive and negative experiences shaped your cultural identity while growing up? (e.g. memories at home, school, university, a work environment that made you feel proud or ashamed of your culture) ',
                             '3. How connected do you currently feel to your cultural identity (roots)?',
                             '4. Did you wish you felt more connected to your cultural identity (roots)?',
                             '5. What did you wish you knew more about your culture?',
                             '6. Would you visit the country of your ethnic origin to learn more about your cultural identity?',
                             '7.  Are there any obstacles or limitations that prevent you from visiting your country of ethnic origin?',
                             "8. Do you think it's important to connect back to your cultural identity (roots)?",
                             "9. Why do you think it's important (or not) to connect back with your cultural identity (roots)?",
                             '10. Suppose there was a cultural reconnection programme that was established in the motherland to help diaspora people reconnect back with their cultural identity. Would you be interested to fly back and participate? (Where you would formally learn about your culture and also work with a local NGO solving a social problem)',
                             '10a. If you selected ‘No’, why would you not want to participate in this programme?',
                             '11. Would you be willing to pay for the programme?',
                             '12. What would you want to gain from a cultural reconnection programme like this? (or what elements would you want the programme to have if you are going to be paying for it?)',
                             '13. How long would you be willing to volunteer with a local NGO in the country of your ethnic origin?',
                             'Let us know if there is anything else you would like to mention below',
                             'CF Total ethnicities (List)',
                             'CF Total ethnicities (Number)',
                             'CF Total countries grown up in (List)',
                             'CF Total countries grown up in (Number)'],
                 var_name='question_wording_legacy',value_name='response')

### Merge un-pivoted raw data with question meta data
Enrich the raw survey data unpivoted in previous step with question meta data (e.g. question type, data type etc.)

In [19]:
# Left join between raw dataset and the Q20 manual categorisation table
df_td = pd.merge(df_piv, df_md, how='left', left_on='question_wording_legacy', right_on='question_wording_legacy')

In [20]:
# Re-order columns in spreadsheet
col = df_td.pop('question_id')
df_td.insert(3, col.name, col)

col = df_td.pop('response')
df_td.insert(9, col.name, col)

In [21]:
# Update question_id and quesetion_wording columns to reflect the newly cleansed Q18 & Q20
df_td['question_id'] = df_td.apply(lambda x: 'Q18C' if x['question_wording_legacy'] == 'Please list the rest of your ethnicities here if you have more than 3 (Cleansed)' else x['question_id'], axis=1)
df_td['question_id'] = df_td.apply(lambda x: 'Q20C' if x['question_wording_legacy'] == 'Please add any other countries you grew up in (Cleansed)' else x['question_id'], axis=1)

df_td['question_wording'] = df_td.apply(lambda x: 'Please add any other countries you grew up in (Cleansed)' if x['question_wording_legacy'] == 'Please list the rest of your ethnicities here if you have more than 3 (Cleansed)' else x['question_wording'], axis=1)
df_td['question_wording'] = df_td.apply(lambda x: 'Please add any other countries you grew up in (Cleansed)' if x['question_wording_legacy'] == 'Please add any other countries you grew up in (Cleansed)' else x['question_wording'], axis=1)

In [22]:
# Update columns to reflect additional concatenation based calculated fields 
df_td['question_id'] = df_td.apply(lambda x: 'QCF' if x['question_wording_legacy'] in ['CF Total ethnicities (List)','CF Total ethnicities (Number)','CF Total countries grown up in (List)','CF Total countries grown up in (Number)'] else x['question_id'], axis=1)
df_td['question_wording'] = df_td.apply(lambda x: x['question_wording_legacy'] if x['question_wording_legacy'] in ['CF Total ethnicities (List)','CF Total ethnicities (Number)','CF Total countries grown up in (List)','CF Total countries grown up in (Number)'] else x['question_wording'], axis=1)

In [23]:
# SENSE CHECK: should = 5600, as 200 responses x (22 qs incl. 10i and 10ii + x2 cleansed qs + 4CFs)
df_td.shape[0]

5600

### Explode calculated field questions 
For questions where multiple countries are specified, explode the data set so we have the data across multiple lines. This will help with analysis to understand how many instances of the country 'Germany' comes up for example. In the current wide data set, it is difficult to do this without a significant amount of manual intervention.

In [24]:
# Filter dataframe for the calc field 'Total ethnicities (list)'
df_td_filtered = df_td.query('question_wording == "CF Total ethnicities (List)"').copy()

# Convert the 'countries' column from a comma-separated string to a list
df_td_filtered['response'] = df_td_filtered['response'].apply(lambda x: x.split(', '))

# Explode response column
df_expl = df_td_filtered.explode('response')

In [25]:
# Filter dataframe for the calc field 'Total ethnicities (list)'
df_td_filtered2 = df_td.query('question_wording == "CF Total countries grown up in (List)"').copy()

# Convert the 'countries' column from a comma-separated string to a list
df_td_filtered2['response'] = df_td_filtered2['response'].apply(lambda x: x.split(', '))

# Explode response column
df_expl2 = df_td_filtered2.explode('response')

In [26]:
# Filter dataframe to exclude exploded responses
df_unexpl = df_td.query('question_wording != ["CF Total ethnicities (List)","CF Total countries grown up in (List)"]')

In [27]:
# Concatenate 'unexploded' dataset with 2 questions that have undergone 'explosion'
df_final = pd.concat([df_unexpl, df_expl, df_expl2])

In [28]:
# Sort final dataframe by recipient_id and question_id
df_td = df_final.sort_values(by=['recipient_id','question_id','question_wording'])

In [29]:
df_td.shape[0]

5777

### Create function to build service
**Reference**: Used the following walkthrough (Section: Write dataframe to Google Sheet) \
https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c

In [30]:
# Create a pickle file from JSON file and build service
def createService(client_secret_file, api_service_name, api_version, *scopes):
    global service
    SCOPES = [scope for scope in scopes[0]]    
    cred = None

    if os.path.exists('token_write.pickle'):
        with open('token_write.pickle', 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
            cred = flow.run_local_server()

        with open('token_write.pickle', 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(api_service_name, api_version, credentials=cred)
        print(api_service_name, 'service created successfully')

    except Exception as e:
        print(e)

In [31]:
# Call function
createService('credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])

sheets service created successfully


### Add new sheet to existing Google Sheet
**Reference**: Used the following walkthrough \
https://learndataanalysis.org/add-new-worksheets-to-existing-google-sheets-file-with-google-sheets-api/

**Note**: Each time this code is run it will create a new tab based on the date and time of running to ensure no data is written over.

In [32]:
# Define Google Sheet parameter
gsheetId = '1YDodYS7JAV7R0icO0s28i9UGKHO2LbOcbfCYjmhvSmI'

# Define variables for new sheet name
date = str(datetime.today().strftime("%Y-%m-%d-%H:%M"))
sheet_name_td = f'{date}-tall-dataset'
sheet_name_wd = f'{date}-wide-dataset'

In [33]:
# Add new sheet to existing Google Sheet
def addSheet(sheet_name):
    request_body = {
            'requests': [{
                'addSheet': {
                    'properties': {
                        'title': sheet_name,
                        'tabColor': {
                            'red': 0.44,
                            'green': 0.99,
                            'blue': 0.50
                        }
                    }
                }
            }]
        }
    
    response_date = service.spreadsheets().batchUpdate(
        spreadsheetId=gsheetId,
        body=request_body
    ).execute()

In [34]:
# Call function
addSheet(sheet_name_td)

In [35]:
# Call function
addSheet(sheet_name_wd)

In [36]:
# Define Google Sheet parameter
exportRange = f'{sheet_name_td}!A1:AA6000'

In [37]:
# Update newly created sheet with engineered dataframe (df)
def exportData(dataframe):
    response_date = service.spreadsheets().values().update(
        spreadsheetId=gsheetId,
        valueInputOption='RAW',
        range=exportRange,
        body=dict(
            majorDimension='ROWS',
            values=dataframe.T.reset_index().T.values.tolist())
    ).execute()

In [38]:
# Fill NaN values
df_td = df_td.fillna('')

# Call function
exportData(df_td)

### Create wide data set with new naming convention
Certain data analysis is prefereable with a wide data set, therefore we will also add a 'wide' data set sheet to our final GSheet.

In [39]:
# Rename column headers to align with Q01, Q02 etc. naming convention outlined above
df_wd = df_rd.rename(columns={
    'recipient_id':'Recipient ID',
    'timestamp':'Timestamp',
    'Age?':'Q14 Age?',
    'What country is your first ethnic background? ':'Q15 What country is your first ethnic background?',
    'What country is your second ethnic background? (If you have multiple)':'Q16 What country is your second ethnic background? (If you have multiple)',
    'What country is your third ethnic background? (If you have multiple)':'Q17 What country is your third ethnic background? (If you have multiple)',
    'Please list the rest of your ethnicities here if you have more than 3':'Q18 Please list the rest of your ethnicities here if you have more than 3',
    'Please list the rest of your ethnicities here if you have more than 3 (Cleansed)':'Q18 Please list the rest of your ethnicities here if you have more than 3 (Cleansed)',
    'What country did you grow up in primarily? ':'Q19 What country did you grow up in primarily?',
    'Please add any other countries you grew up in':'Q20 Please add any other countries you grew up in',
    'Please add any other countries you grew up in (Cleansed)':'Q20 Please add any other countries you grew up in (Cleansed)', 
    '1. Did you experience uncertainty or conflict between your family culture and the culture of the country you grew up in?':'Q01 Did you experience uncertainty or conflict between your family culture and the culture of the country you grew up in?',
    '2.  What kind of positive and negative experiences shaped your cultural identity while growing up? (e.g. memories at home, school, university, a work environment that made you feel proud or ashamed of your culture) ':'Q02  What kind of positive and negative experiences shaped your cultural identity while growing up? (e.g. memories at home, school, university, a work environment that made you feel proud or ashamed of your culture)',
    '3. How connected do you currently feel to your cultural identity (roots)?':'Q03 How connected do you currently feel to your cultural identity (roots)?',
    '4. Did you wish you felt more connected to your cultural identity (roots)?':'Q04 Did you wish you felt more connected to your cultural identity (roots)?',
    '5. What did you wish you knew more about your culture?':'Q05 What did you wish you knew more about your culture?',
    '6. Would you visit the country of your ethnic origin to learn more about your cultural identity?':'Q06 Would you visit the country of your ethnic origin to learn more about your cultural identity?',
    '7.  Are there any obstacles or limitations that prevent you from visiting your country of ethnic origin?':'Q07 Are there any obstacles or limitations that prevent you from visiting your country of ethnic origin?',
    "8. Do you think it's important to connect back to your cultural identity (roots)?":"Q08 Do you think it's important to connect back to your cultural identity (roots)?",
    "9. Why do you think it's important (or not) to connect back with your cultural identity (roots)?":"Q09 Why do you think it's important (or not) to connect back with your cultural identity (roots)?",
    '10. Suppose there was a cultural reconnection programme that was established in the motherland to help diaspora people reconnect back with their cultural identity. Would you be interested to fly back and participate? (Where you would formally learn about your culture and also work with a local NGO solving a social problem)':'Q10i Suppose there was a cultural reconnection programme that was established in the motherland to help diaspora people reconnect back with their cultural identity. Would you be interested to fly back and participate? (Where you would formally learn about your culture and also work with a local NGO solving a social problem)',
    '10a. If you selected ‘No’, why would you not want to participate in this programme?':'Q10ii If you selected ‘No’, why would you not want to participate in this programme?',
    '11. Would you be willing to pay for the programme?':'Q11 Would you be willing to pay for the programme?',
    '12. What would you want to gain from a cultural reconnection programme like this? (or what elements would you want the programme to have if you are going to be paying for it?)':'Q12 What would you want to gain from a cultural reconnection programme like this? (or what elements would you want the programme to have if you are going to be paying for it?)',
    '13. How long would you be willing to volunteer with a local NGO in the country of your ethnic origin?':'Q13 How long would you be willing to volunteer with a local NGO in the country of your ethnic origin?',
    'email':'Email',
    'Let us know if there is anything else you would like to mention below':'Q21 Let us know if there is anything else you would like to mention below'})

In [40]:
# Sort dataframe headers alphabetically to ensure Q01, Q02 etc. are appearing in the correct order
df_wd = df_wd.sort_index(axis=1)

In [41]:
# Re-order columns 'Recipient ID' and 'Timestamp' in spreadsheet
col = df_wd.pop('Recipient ID')
df_wd.insert(0, col.name, col)

col = df_wd.pop('Timestamp')
df_wd.insert(1, col.name, col)

In [42]:
# Fill NaN values
df_wd = df_wd.fillna('')

In [43]:
# Define Google Sheet parameter
exportRange = f'{sheet_name_wd}!A1:AE5000'

In [44]:
# Call function
exportData(df_wd)